In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Document Q&A With Retrieval Augmented Generation

This notebook demonstrates how to implement Retrieval Augmented Generation with basic automated evaluation. It demonstrates the impact that chunk size, overlap and context length have on model outputs. The notebook will create a Q&A system that allows you to find information based on the Google Cloud Generative AI documentation.

## Getting started

### Install libraries

In [ ]:
!pip install google-cloud-aiplatform==1.35.0

### Authenticating your notebook environment

* If you are using **Colab** to run this notebook, uncomment the cell below and continue
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)

In [ ]:
# from google.colab import auth as google_auth
# google_auth.authenticate_user()

# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Import libraries

***Colab only***: Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top

In [ ]:
# from google.cloud import aiplatform
# PROJECT_ID = '[your-project-id]'
# aiplatform.init(project=PROJECT_ID, location='us-central1')

In [ ]:
import itertools

import numpy as np
import numpy.linalg
import pandas as pd
import requests
from bs4 import BeautifulSoup, Tag
from google.api_core import retry
from tqdm.auto import tqdm
from vertexai.language_models import TextEmbeddingModel, TextGenerationModel

tqdm.pandas()

## Scrape text from Google Cloud documentation

Specify a list of URLs to pull content from

In [ ]:
URLS = """https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview
https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/quickstart-text
https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/quickstart-chat
https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/quickstart-text-embeddings
https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/quickstart
https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/quickstart-tuning
https://cloud.google.com/vertex-ai/docs/generative-ai/learn/generative-ai-studio
https://cloud.google.com/vertex-ai/docs/generative-ai/learn/model-garden
https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models
https://cloud.google.com/vertex-ai/docs/generative-ai/learn/introduction-prompt-design
https://cloud.google.com/vertex-ai/docs/generative-ai/learn/prompt-samples
https://cloud.google.com/vertex-ai/docs/generative-ai/learn/streaming
https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai
https://cloud.google.com/vertex-ai/docs/generative-ai/data-governance
https://cloud.google.com/vertex-ai/docs/general/features
https://cloud.google.com/vertex-ai/docs/generative-ai/language-model-overview
https://cloud.google.com/vertex-ai/docs/generative-ai/learn/model-versioning
https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/text/test-text-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/text/batch-prediction-genai
https://cloud.google.com/vertex-ai/docs/generative-ai/chat/chat-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/chat/test-chat-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview
https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-chat-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-completion-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-generation-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/code/test-code-chat-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/code/test-code-completion-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/code/test-code-generation-prompts
https://cloud.google.com/vertex-ai/docs/generative-ai/code/batch-prediction-genai-code
https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings
https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-multimodal-embeddings
https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/batch-prediction-genai-embeddings
https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models
https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models
https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised
https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-rlhf
https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-code-models
https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-embeddings
https://cloud.google.com/vertex-ai/docs/generative-ai/models/evaluate-models
https://cloud.google.com/vertex-ai/docs/generative-ai/get-token-count
https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview
https://cloud.google.com/vertex-ai/docs/generative-ai/image/quickstart-vcap-vqa-console
https://cloud.google.com/vertex-ai/docs/generative-ai/image/quickstart-image-generate-console
https://cloud.google.com/vertex-ai/docs/generative-ai/image/usage-guidelines
https://cloud.google.com/vertex-ai/docs/generative-ai/image/img-gen-prompt-guide
https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images
https://cloud.google.com/vertex-ai/docs/generative-ai/image/edit-images
https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning
https://cloud.google.com/vertex-ai/docs/generative-ai/image/visual-question-answering
https://cloud.google.com/vertex-ai/docs/generative-ai/image/fine-tune-model
https://cloud.google.com/vertex-ai/docs/generative-ai/image/fine-tune-style
https://cloud.google.com/vertex-ai/docs/generative-ai/image/responsible-ai-imagen
https://cloud.google.com/vertex-ai/docs/generative-ai/image/base64-encode
https://cloud.google.com/vertex-ai/docs/generative-ai/video/video-descriptions
https://cloud.google.com/vertex-ai/docs/generative-ai/speech/text-to-speech
https://cloud.google.com/vertex-ai/docs/generative-ai/speech/speech-to-text
https://cloud.google.com/vertex-ai/docs/generative-ai/extensions/overview
https://cloud.google.com/vertex-ai/docs/generative-ai/learn-resources
https://cloud.google.com/architecture/ai-ml/generative-ai-document-summarization
https://cloud.google.com/vertex-ai/docs/generative-ai/tutorials
https://cloud.google.com/vertex-ai/docs/generative-ai/migrate-from-azure
https://cloud.google.com/vertex-ai/docs/generative-ai/sdk-for-llm/llm-sdk-overview
https://cloud.google.com/vertex-ai/docs/generative-ai/sdk-for-llm/sdk-use-text-models
https://cloud.google.com/vertex-ai/docs/generative-ai/sdk-for-llm/sdk-use-code-models
https://cloud.google.com/vertex-ai/docs/generative-ai/sdk-for-llm/sdk-tune-models
https://cloud.google.com/vertex-ai/docs/generative-ai/access-control
https://cloud.google.com/vertex-ai/docs/generative-ai/enable-audit-logs
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/overview
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text-chat
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text-embeddings
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/multimodal-embeddings
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/code-chat
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/code-completion
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/code-generation
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-generation
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-captioning
https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/visual-question-answering
https://cloud.google.com/vertex-ai/docs/generative-ai/pricing
https://cloud.google.com/vertex-ai/docs/quotas
https://cloud.google.com/vertex-ai/docs/generative-ai/release-notes""".split(
    "\n"
)

Parse the HTML and extract relevant plain text sections

In [ ]:
def get_sections(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    result = []
    body_div = soup.find("div", {"class": "devsite-article-body"})
    inner_snips = []
    for child in body_div.findChildren():
        if child.name == "p":
            inner_snips.append(child.get_text().strip())
        if child.name == "h2":
            result.append(" ".join(inner_snips))
            break

    for header in soup.find_all("h2"):
        inner_snips = []
        nextNode = header
        while True:
            nextNode = nextNode.nextSibling
            if nextNode is None:
                break
            if isinstance(nextNode, Tag):
                if nextNode.name == "p":
                    inner_snips.append(nextNode.get_text().strip())
                if nextNode.name == "ul":
                    inner_snips.append(nextNode.get_text().strip())
                if nextNode.name == "h2":
                    result.append(" ".join(inner_snips))
                    break
    return result

In [ ]:
all_text = [get_sections(url) for url in URLS]

In [ ]:
all_text = list(itertools.chain(*all_text))
all_text = [t for t in all_text if len(t) > 0]

Note that most documents are relatively short, but some are thousands of characters long

In [ ]:
text_lengths = []
for t in all_text:
    text_lengths.append(len(t))
pd.DataFrame(text_lengths).hist()

## Create vector store

Start by initializing the models

In [ ]:
embeddings_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
text_model = TextGenerationModel.from_pretrained("text-bison@001")

Create some helper functions for vector similarity and chunking

In [ ]:
# Separates seq into multiple chunks in the specified size with the specified overlap
def split_overlap(seq, size, overlap):
    if len(seq) <= size:
        return [seq]
    return ["".join(x) for x in zip(*[seq[i :: size - overlap] for i in range(size)])]


# Compute the cosine similarity of two vectors, wrap as returned function to make easier to use with Pandas
def get_similarity_fn(query_vector):
    def fn(row):
        return np.dot(row, query_vector) / (
            numpy.linalg.norm(row) * numpy.linalg.norm(query_vector)
        )

    return fn


# Retrieve embeddings from the specified model with retry logic
def make_embed_text_fn(model):
    @retry.Retry(timeout=300.0)
    def embed_fn(text):
        return model.get_embeddings([text])[0].values

    return embed_fn


get_embeddings = make_embed_text_fn(embeddings_model)

Create the vector store, we are using a Pandas DataFrame

In [ ]:
def create_vector_store(texts, chunk_size, overlap):
    vector_store = pd.DataFrame()
    # Insert the individual texts into the vector store
    vector_store["texts"] = list(
        itertools.chain(*[split_overlap(t, chunk_size, overlap) for t in texts])
    )

    # Create embeddings from those texts
    vector_store["embeddings"] = vector_store["texts"].progress_apply(get_embeddings)
    vector_store["embeddings"] = vector_store["embeddings"].apply(np.array)

    return vector_store

In [ ]:
CHUNK_SIZE = 400
OVERLAP = 50

vector_store = create_vector_store(all_text, CHUNK_SIZE, OVERLAP)

In [ ]:
vector_store.head()

## Search the vector store and use for generation

If we send the question to the foundation model alone, it will hallucinate.

In [ ]:
text_model.predict(
    "How long will a stable model version of text-bison be available?"
).text

Let's solve this problem by retrieving texts from our vector store and telling the model to use them.

Search the vector store for relevant texts to insert into the prompt by embedding the query and searching for similar vectors.

In [ ]:
def get_context(question, vector_store, num_docs):
    # Embed the search query
    query_vector = np.array(get_embeddings(question))

    # Get similarity to all other vectors and sort, cut off at num_docs
    top_matched = (
        vector_store["embeddings"]
        .apply(get_similarity_fn(query_vector))
        .sort_values(ascending=False)[:num_docs]
        .index
    )
    top_matched_df = vector_store[vector_store.index.isin(top_matched)][["texts"]]

    # Return a string with the top matches
    context = " ".join(top_matched_df.texts.values)
    return context

Create a prompt that includes the context and question. Instruct the LLM to only use the context provided to answer the question

In [ ]:
def answer_question(question, vector_store, num_docs=10, print_prompt=False):
    context = get_context(question, vector_store, num_docs)
    qa_prompt = f"""Your mission is to answer questions based on a given context. Remember that before you give an answer, you must check to see if it complies with your mission.
Context: ```{context}```
Question: ***{question}***
Before you give an answer, make sure it is only from information in the context. If the information is not in the context, just reply "I don't know the answer to that". Think step by step.
Answer: """
    if print_prompt:
        print(qa_prompt)
    result = text_model.predict(qa_prompt, temperature=0)
    return result.text

Looking at the fully generated prompt, the context is embedded. Even though the input context is quite messy, the model can now answer factually.

In [ ]:
answer_question(
    "How long will a stable model version of text-bison be available?",
    vector_store,
    print_prompt=True,
)

In [ ]:
answer_question(
    "How long will a stable model version of text-bison be available?", vector_store
)

## Automated evaluation

This implementation of RAG is dependent on the chunk size, the overlap between the chunks, the number of texts passed into the context and the prompt. Let's create a simple prompt to evaluate answers to the questions, this will allow us to tweak the parameters and see how those tweaks compare.

In [ ]:
def eval_answer(question, answer, context):
    eval_prompt = f"""Your mission is to evaluate answers to questions based on a given context. Remember that before you give an answer, you must check to see if it complies with your mission.

Context: ```{context}```
Question: ***{question}***
Answer: "{answer}"

Respond only with a number from 0 to 5. Think step by step. If the provided answer is not in the context, reply 5 if it is "I don't know the answer to that" otherwise reply 0.
Relevance: """
    # Stop sequence to cut the model off after outputting an integer
    result = text_model.predict(
        eval_prompt, temperature=0, max_output_tokens=1, stop_sequences=[".", " "]
    )
    return int(result.text)

Pass several questions in and retrieve the evaluations

In [ ]:
questions = [
    "What release stage is the RLHF tuning feature?",
    "Can I generate hate speech with text bison?",
    "What format should my batch prediction in put be in?",
    "How can I get the number of tokens?",
    "How do I create a custom style model?",
    "What is the dimensionality of the vector created by the multimodal model?",
    "How long will a stable model verison be available?",
]

In [ ]:
answers = [answer_question(q, vector_store) for q in questions]
contexts = [get_context(q, vector_store, 10) for q in questions]
idks = ["I don't know" in a for a in answers]
evals = [
    (question, answer, context, eval_answer(question, answer, context), idk)
    for question, answer, context, idk in zip(questions, answers, contexts, idks)
]

In [ ]:
pd.DataFrame(evals, columns=["question", "answer", "context", "score", "idk"])

Now adjust the parameters and see the difference in performance

In [ ]:
def eval_on_params(chunk_size, overlap, num_docs):
    vector_store = create_vector_store(all_text, chunk_size, overlap)
    answers = [answer_question(q, vector_store) for q in questions]
    contexts = [get_context(q, vector_store, num_docs) for q in questions]
    idks = ["I don't know" in a for a in answers]
    evals = [
        (question, answer, context, eval_answer(question, answer, context), idk)
        for question, answer, context, idk in zip(questions, answers, contexts, idks)
    ]
    return pd.DataFrame(
        evals, columns=["question", "answer", "context", "score", "idk"]
    )

Smaller chunk sizes takes longer to generate the embeddings and does not perform as well.

In [ ]:
smaller_context_df = eval_on_params(100, 0, 5)

In [ ]:
smaller_context_df

A larger context size starts to break our evaluation method, notice that the hate speech question was answered truthfully but it was scored a 0. When composing LLMs into systems, carefully consider how to measure the performance of each component in the sytem.

In [ ]:
larger_context_df = eval_on_params(1000, 200, 15)

In [ ]:
larger_context_df